# Tutorial on CasADi and Opti
For more information on CasADi, check out https://web.casadi.org/docs/

For IPOPT options, check out https://coin-or.github.io/Ipopt/OPTIONS.html

In [ ]:
from casadi import *
import matplotlib.pyplot as plt
from matplotlib.animation import *
from matplotlib.patches import Circle
from IPython.display import HTML

In [ ]:
def plotObjective():
    plt.figure()

    delta = 0.025
    xx = np.arange(-1.7, 1.7, delta)
    yy = np.arange(-1.6, 1.6, delta)
    X, Y = np.meshgrid(xx, yy)
    Z = (1-X)**2 + (Y-X**2)**2
    plt.contour(X, Y, Z, levels=100)

    plt.gca().set_xlim([-1.7, 1.7])
    plt.gca().set_ylim([-1.5, 1.5])
    plt.gca().set_aspect('equal')

def createAnimation(opti_f):
    xx_sol = []
    yy_sol = []
    slopes = np.linspace(-2.0, 2.0, 30)
    for slope in slopes:
        x, y = opti_f(slope)
        xx_sol.append(x)
        yy_sol.append(y)
    
    plotObjective()
    plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
             np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4)
    constraint, = plt.plot(np.linspace(-2.0, 2.0, 2), slopes[0]*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
    solution, = plt.plot(xx_sol[0], yy_sol[0], marker='o', color='k', markersize=10)
    
    def update_frame(i):
        constraint.set_data(np.linspace(-2.0, 2.0, 2), slopes[i]*np.linspace(-2.0, 2.0, 2))
        solution.set_data([float(xx_sol[i])], [float(yy_sol[i])])
        return constraint, solution,
    
    animation1 = FuncAnimation(plt.gcf(), update_frame, frames=range(len(slopes)))
    
    HTML(animation1.to_jshtml())
    return animation1

## Basic CasADi introduction: variables and functions

Lets define our first CasADi function:

$f\left( \begin{bmatrix} x_1 & x_2 & x_3 & x_4 & x_5 \end{bmatrix}^\top \right) = \begin{bmatrix} x_1 + x_2 + x_3\\ x_3^2 + x_4^2 + x_5^2\end{bmatrix}$

In [ ]:
# Define x and y


# Assign the values


# Define a function and evaluate it



Lets introduce a paremeter in our CasADi function:

$f\left( \begin{bmatrix} x_1 & x_2 & x_3 & x_4 & x_5 \end{bmatrix}^\top, \textcolor{blue}{p} \right) = \begin{bmatrix} x_1 + x_2 + x_3\\ x_3^2 + x_4^2 + x_5^2 + \textcolor{blue}{p}\end{bmatrix}$

In [ ]:
# Define x and y


# Define p


# Assign values


# Define the function and print it
f = Function("f", [x], [y])
print(f([1, 2, 3, 4, 5]))

# What happens if y is changed?

f = Function("f", [x], [y])
print(f([1, 2, 3, 4, 5]))

## Using CasADi's Algorithmic differentation

Computing a derivative using jacobian:

In [ ]:


print(J)
print(J([1,2,3,4,5]))

Computing a second order derivative using hessian:

In [ ]:


print(H)
print(H([1,2,3,4,5]))

## Storing, loading and code generating casadi functions

In [ ]:
# loading a casadi function
# TODO

# saving a casadi function
# TODO

# code generation
# TODO

# compiling code-generated casadi functions
# gcc -fPIC -shared myFunction.c -o myFunction.so

## Opti example
(see https://web.casadi.org/blog/opti/)
Let's now define an optimization problem and solve it using CasADi Opti.

### Basic Rosenbrock example
\begin{align}
\min_{x,u} \quad (1-x)^2 + (y-x^2)^2
\end{align}

In [ ]:
# create an opti instance
# TODO

# define optimization variables
# TODO

# define the objective function
# TODO

# set the solver
# TODO

# show the output
print(f"Optimal solution: ({}, {})")

plotObjective()
plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

### Let's add a simple constraint
\begin{align}
\min_{x,u} (1-x)^2 + (y-x^2)^2\\
\text{s.t.} \quad x^2 + y^2 = 1\quad 
\end{align}

In [ ]:
# define the constraint
# TODO

# set the solver
opti.solver("ipopt", {}, {})
sol = opti.solve()

# show the solution
print(f"Optimal solution: ({sol.value(x)}, {sol.value(y)})")
print(f"Lagrange multiplier value: {sol.value(opti.debug.lam_g)}")

plotObjective()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
         np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

### Let's add an inequality constraint also
\begin{align}
\min_{x,u} (1-x)^2 + (y-x^2)^2\\
\text{s.t.} \quad x^2 + y^2 = 1\\
y >= px
\end{align}

In [ ]:
# define a parameter
# TODO

# define the constraints
# TODO

# set the parameter value
# TODO

# solve the problem
sol = opti.solve()

# show the solution
print(f"Optimal solution: ({sol.value(x)}, {sol.value(y)})")

plotObjective()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
         np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
plt.plot(sol.value(x), sol.value(y), 'k', marker='o', markersize=10)

## More advanced options in Opti
#### Callbacks

In [ ]:
# set the solver
opti.solver("ipopt", {"print_time":False}, {"print_level":0, "max_iter":3000})

# define a callback
def plotAll():
    plotObjective()
    plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
    plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
    plt.plot(opti.debug.value(x), opti.debug.value(y), 'k', marker='o', markersize=10)

opti.callback(lambda i : print(opti.debug.value(x), opti.debug.value(y)))
# opti.callback(lambda i : plotAll())

# solve the problem
opti.solve()

#### opti.to_function

In [ ]:
# create function object
# TODO

# solve the problem
# TODO

# show the solution
print(f"Optimal solution: ({sol_x}, {sol_y})")

plotObjective()
plt.plot(np.cos(np.linspace(0, 2*np.pi, 100)), 
         np.sin(np.linspace(0, 2*np.pi, 100)), 'r', linewidth=4.0)
plt.plot(np.linspace(-2.0, 2.0, 2), 2.0*np.linspace(-2.0, 2.0, 2), "r", linewidth=4)
plt.plot(sol_x, sol_y, 'k', marker='o', markersize=10)

In [ ]:
animation1 = createAnimation(opti_f)
HTML(animation1.to_jshtml())

### initial guesses

In [ ]:
# set an initial guess ((x,y) = (0.5, 2.0))
# TODO

# create function object
opti_f = opti.to_function("opti_f", [p], [x, y])

animation2 = createAnimation(opti_f)
HTML(animation2.to_jshtml())

# Example problem: hanging chain

Now, lets consider a more elaborate example optimization problem. We want to find the shape of a chain, fixed on both ends. This can be achieved by representing the chain by masses attached to each other with simple springs. Every mass has a coordinate $(x_i, y_i)$ and a mass $m_i$.

To find the shape of the chain, we minimize the sum of the potential energy in all springs and the gravitational potential energy of all masses. This results in the cost function

$J = \frac{1}{2}\sum_{i=1}^{N-1}{D_i \left(\sqrt{(x_i - x_{i+1})^2 + (y_i - y_{i+1})^2} -\frac{L}{N}\right)^2} + g\sum_{i=1}^{N}{m_i y_i}$

where $D_i = 70N [N/m]$ is a spring constant, $L = 2.0m$ is the length of the chain at rest (if all springs are at their rest length), $g = 9.81 [m/s^2]$ is the gravitational constant. You can use $m_i = 0.1 [kg]$, but feel free to play around with this value (or use different masses for different parts of the chain).

One side of the chain is attached to the point $(-1, 1)$ and the other side is attached to the point $(1, 1)$.

There are also some obstacles present through which the chain cannot go

a) Solve the optimization problem to find the shape of the chain

b) Can you make the chain rest on top of the highest obstacle?

In [ ]:
class obstacle:
    def __init__(self, center_x, center_y, radius):
        self.center_x = center_x
        self.center_y = center_y
        self.radius = radius

obstacles = [obstacle(-0.7, 0.7, 0.1), 
             obstacle(0.1, 0.8, 0.1), 
             obstacle(0.8, 0.8, 0.1),
             obstacle(-0.5, 1.2, 0.1),
             obstacle(0.0, 1.5, 0.1)]


# Write your optimization code here
#
# The plotting code below expects an opti_function object called 'opti_chain' 
# that takes two arguments:
#      - L: the length of the chain
#      - y: an initial guess for the height of the chain
#
# The function outputs the x- and y-coordinates of the chain masses
#
#

plt.figure()
x_chain, y_chain = opti_chain(2.0, 1.0)
for obs in obstacles:
    plt.gca().add_patch(Circle((obs.center_x, obs.center_y), obs.radius, color='r', alpha=0.5))
plt.plot(x_chain, y_chain, 'b-o')
plt.plot(p_left[0], p_left[1], 'ko')
plt.plot(p_right[0], p_right[1], 'ko')
plt.axis('equal')
plt.show()